# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint 


# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
expcsv = pd.read_csv("weather.csv")
weather_df = pd.DataFrame(expcsv)
weather_df


#the dates are in time stamps, i found this online as an example:
#import datetime
#timestamp = datetime.datetime.fromtimestamp(1500000000)
#print(timestamp.strftime('%Y-%m-%d %H:%M:%S'))

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Avarua,-21.21,-159.78,77.000,78,100,8.70,CK,1595295422
1,Ushuaia,-54.80,-68.30,28.400,100,0,2.60,AR,1595295077
2,Pocerski Pričinović,44.72,19.71,66.200,77,0,2.10,RS,1595295422
3,Hasaki,35.73,140.83,82.994,88,75,2.60,JP,1595295422
4,Rikitea,-23.12,-134.97,67.352,69,82,7.62,PF,1595295422
...,...,...,...,...,...,...,...,...,...
602,İmamoğlu,37.27,35.66,78.998,100,75,2.60,TR,1595295460
603,La Ronge,55.10,-105.28,68.000,68,75,2.60,CA,1595295460
604,Jericó,5.79,-75.79,62.600,88,75,3.10,CO,1595295460
605,Umluj,25.02,37.27,85.280,60,40,3.96,SA,1595295460


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key = g_key)
locations = weather_df[["Lat","Lng"]].astype(float)
weight = weather_df["Humidity"].astype(int)

In [5]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weight,dissipating=False, max_intensity=100,point_radius = 3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [10]:
vacation = weather_df[weather_df["Max Temp"]<80]
vacation = vacation[vacation["Max Temp"]>70]
vacation = vacation[vacation["Wind Speed"]<10]
vacation = vacation[vacation["Cloudiness"]==0]
vacation.reset_index(drop=True, inplace=True)
vacation.head(10)


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Yangi Marg`ilon,40.43,71.72,73.400,40,0,4.10,UZ,1595295422
1,Karratha,-20.74,116.85,71.186,42,0,7.30,AU,1595295430
2,Menzel Salem,35.86,8.48,73.400,46,0,2.10,TN,1595295433
3,Arraial do Cabo,-22.97,-42.03,70.610,85,0,8.33,BR,1595295439
4,São João da Barra,-21.64,-41.05,70.736,88,0,5.02,BR,1595295440
5,Benghazi,32.12,20.07,76.064,71,0,7.64,LY,1595295440
6,Aksarayskiy,46.79,48.01,78.008,73,0,2.08,RU,1595295441
7,Tataouine,32.93,10.45,71.600,73,0,2.60,TN,1595295442
8,Cururupu,-1.83,-44.87,73.508,88,0,0.96,BR,1595295443
9,Léo,11.10,-2.10,73.364,85,0,2.05,BF,1595295444


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
hotel_df = vacation[["City", "Country", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Date"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Hotel Name
0,Yangi Marg`ilon,UZ,40.43,71.72,73.400,40,0,4.10,1595295422,
1,Karratha,AU,-20.74,116.85,71.186,42,0,7.30,1595295430,
2,Menzel Salem,TN,35.86,8.48,73.400,46,0,2.10,1595295433,
3,Arraial do Cabo,BR,-22.97,-42.03,70.610,85,0,8.33,1595295439,
4,São João da Barra,BR,-21.64,-41.05,70.736,88,0,5.02,1595295440,


In [12]:
#used API day 3, exam restaurant example where they did something similar

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {  
    "radius": "5000",
    "type": "Lodging",
    "key": g_key,}

for index, row in hotel_df.iterrows():
    
    location = row[["Lat","Lng"]]
    
    #used f function because i needed a dataframe to be read as a string
    params['location'] = f"{row['Lat']}, {row['Lng']}"
 
    #assemble url
    response = requests.get(base_url, params=params).json()
    #print(response['status'])
    hotel_df.loc[index,"Hotel Name"] = response['results'][1]['name']
    hotel_df.head()



In [16]:

hotel_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Hotel Name
0,Yangi Marg`ilon,UZ,40.43,71.72,73.400,40,0,4.10,1595295422,ASAKA BANK Margilan
1,Karratha,AU,-20.74,116.85,71.186,42,0,7.30,1595295430,ibis Styles Karratha
2,Menzel Salem,TN,35.86,8.48,73.400,46,0,2.10,1595295433,mohamed ben khlil omri
3,Arraial do Cabo,BR,-22.97,-42.03,70.610,85,0,8.33,1595295439,Pousada Porto Praia
4,São João da Barra,BR,-21.64,-41.05,70.736,88,0,5.02,1595295440,AC São João da Barra
5,Benghazi,LY,32.12,20.07,76.064,71,0,7.64,1595295440,Aldasra Oil Services
6,Aksarayskiy,RU,46.79,48.01,78.008,73,0,2.08,1595295441,Sberbank
7,Tataouine,TN,32.93,10.45,71.600,73,0,2.60,1595295442,Sangho Privilege Tataouine
8,Cururupu,BR,-1.83,-44.87,73.508,88,0,0.96,1595295443,Pousada Través'cia
9,Léo,BF,11.10,-2.10,73.364,85,0,2.05,1595295444,Hotel Sissilis


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_locations = hotel_df[["Lat", "Lng"]]


In [15]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weight,dissipating=False, max_intensity=100,point_radius = 3)
marker_layer = gmaps.marker_layer(hotel_locations, info_box_content = hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
fig
# Display figure


Figure(layout=FigureLayout(height='420px'))